In [2]:
import pandas as pd
import numpy as np

# Download from https://www.pxweb.bfs.admin.ch/pxweb/de/px-x-1703030000_101/px-x-1703030000_101/px-x-1703030000_101.px
csv_name = "original_data_by_municipality.csv"

df = pd.read_csv(csv_name, sep=",", encoding="ISO-8859-1")



df["Original_Order"] = np.arange(len(df))
df = df.sort_values(by=["Datum und Vorlage", "Original_Order"])
df = df.drop("Original_Order", axis=1)

print(df.head())

df.rename(
    columns={"Kanton (-) / Bezirk (>>) / Gemeinde (......)": "Gemeinde"}, inplace=True
)

df["Kanton"] = np.nan
df["Bezirk"] = np.nan

current_kanton = "Test Kanton"
current_bezirk = "Test Bezirk"

for index, row in df.iterrows():
    if row["Gemeinde"].startswith("-"):
        current_kanton = row["Gemeinde"][1:].strip()
    elif row["Gemeinde"].startswith(">>"):
        current_bezirk = row["Gemeinde"][2:].strip()
    elif row["Gemeinde"].startswith("......"):

        df.at[index, "Kanton"] = current_kanton
        df.at[index, "Bezirk"] = current_bezirk

gemeinde_df = df[df["Gemeinde"].str.startswith("......")].copy()

gemeinde_df["Gemeinde"] = gemeinde_df["Gemeinde"].map(lambda x: x[6:].strip())


gemeinde_df["Ja in %"] = pd.to_numeric(gemeinde_df["Ja in %"], errors="coerce")


pivot_ja = gemeinde_df.pivot_table(
    index=["Gemeinde", "Kanton", "Bezirk"],
    columns="Datum und Vorlage",
    values="Ja in %",
)

pivot_ja.to_csv("cleaned_data_by_municipality.csv")

print(df.head())

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [11]:
import pandas as pd
import numpy as np

df = pd.read_csv("cleaned_data_by_municipality.csv")
df_municipalities = pd.read_csv("regions_municipalities.csv", sep=";")
# Match the GEO_NAME of df_municipalities to Gemeinde of df and add a column called Sprachregion to df based on DESC_VAL of df_municipalities

for i, row in df.iterrows():
        df_filtered = df_municipalities[df_municipalities["GEO_NAME"] == row["Gemeinde"]]
        if len(df_filtered) > 0:
            df.at[i, "Sprachregion"] = df_filtered["DESC_VAL"].values[0]
        else:
            df.at[i, "Sprachregion"] = "Unknown"
            print(row["Gemeinde"])

df.to_csv("cleaned_data_by_municipality.csv", index=False)


AG-Ausland-CH
AI-Anderes
AI-Ausland-CH
AI-Korrespondenzweg
BE-Anderes
BS-Ausland-CH
Basse-Vendline
Bois-d'Amont
Damphreux-Lugnez
FR-CH de l'étranger
GE-CH de l'étranger
GE-autres
GR-Anderes
Herznach-Ueken
LU-Ausland-CH
NW-Anderes
Obersaxen Mundaun
SG-Ausland-CH
SO-Anderes
SZ-Anderes
SZ-Korrespondenzweg
TG-Ausland-CH
TI-altri
TI-voto per corrispondenza
UR-Ausland-CH
VD-CH de l'étranger
VD-autres
VS-CH de l'étranger
VS-autres
ZH-Anderes
ZH-Ausland-CH
